In [ ]:
# beat the drum 
# wrote by Jinxin Dong 
# 2018. 11. 15
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

In [4]:
import numpy as np
import cv2
import pygame
import time
import glob
import math
import sys
#import graphics

class Playdrum:
    def __init__(self):
        # area threshold, used to judge whether the drum been hit or not
        self.area_lim =1000
        self.area_lim2 = 600
        pygame.init()
        # game start time 
        self.start_time = time.time()
        self.first_num = 0
        self.second_num = 0
        self.mask_h = 100
        self.mask_w = 100
        self.take_out = 0
        self.hit_on_count = 0
        self.hit_side_count = 0
        # record the hit time (on drum)
        self.p_time = time.time()
        self.n_time = time.time()
        # record the hit time (drum side)
        self.p_side_time = time.time()
        self.n_side_time = time.time()
        
    def play_sound(self,sound_path):
    # TODO: play the music 
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        #print ('playsound!')
        
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick_on(self,image_cut):
    # TODO: detcet the stick hit directly on the drum 
        cv2.imshow('stick on',image_cut)
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        if max_area>4000:
            max_area = 0
        print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            #print (time_diff)
            #print (self.hit_on_count)
            if time_diff> 0.4:
                if self.hit_on_count > 2:
                    self.play_sound("drumbeats/drum.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def detect_stick_side(self,image_cut):
    # TODO: detect the stick hit the drum side
    
        image_cut = image_cut.astype('uint8')
        cv2.imshow('stick side',image_cut.astype('uint8'))
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        #if max_area>4000:
        #    max_area = 0
        print (max_area)
        if max_area>self.area_lim2:
            self.n_side_time = time.time()
            self.hit_side_count  = self.hit_side_count+1
            time_diff = self.n_side_time - self.p_side_time
            #print (time_diff)
            #print (self.hit_side_count)
            if time_diff> 0.3:
                if self.hit_side_count > 2:
                    self.play_sound("drumbeats/drum_loud.wav")
                    self.hit_side_count = 0
                    self.p_side_time = time.time()
        return image_cut
    
    def moment_opencv(self,labelled_in):
        # create object moment 
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res
    def createCircularMask(self,h, w, center=None, radius=None):
        # create the mask with a empty circle inside 
        if center is None:
            center = [int(w/2), int(h/2)]
        if radius is None:
            radius = min(center[0], center[1], w-center[0], h-center[1])

        Y, X = np.ogrid[:h, :w]
        dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

        mask = dist_from_center <= radius
        return mask
    
    def run(self):
        # run main fuction 
        cap = cv2.VideoCapture(0)
        count = 0
        prievous_thresh_original = 0
        prievous_thresh_cut = 0
        scaler = 1.2
        count = 1
        # create the mask
        mask =  self.createCircularMask(self.mask_h, self.mask_w, center=(self.mask_h/2,self.mask_w/2), radius=40)
        mask = np.array(abs(1-mask))
        #cv2.imshow('frame11',mask.astype('uint8'))
        #print (type(mask))
        #print (mask.shape)
        #print (mask)
        
        while(True):
            try:
                
                ret, frame = cap.read()
                output = frame
                gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                # get the drum's location
                circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
                
                # ensure at least some circles were found
                if circles is not None:
                    # convert the (x, y) coordinates and radius of the circles to integers
                    circles = np.round(circles[0, :]).astype("int")

                    # loop over the (x, y) coordinates and radius of the circles
                    for (x, y, r) in circles:
                        # draw the circle in the output image, then draw a rectangle
                        # corresponding to the center of the circle
                        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                        par = np.int64(scaler*r)
                        retval, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
                        thresh_cut = cv2.resize(thresh[y-par:y+par,x-par:x+par],(self.mask_h,self.mask_w))
                        thresh_cut_inv = cv2.bitwise_not(thresh_cut)
                        thresh_original = cv2.resize(thresh[y-r:y+r,x-r:x+r],(self.mask_h,self.mask_w))
                        #cv2.imshow('frame4',thresh_cut)

                        if count !=1:
                            thresh1 = self.detect_stick_side((thresh_cut-previous_thresh_cut)*mask)
                            #thresh1 = self.detect_stick_side(thresh_cut)
                            thresh2 = self.detect_stick_on(thresh_original-previous_thresh_original)

                        #cv2.imshow('frame3',thresh_original)
                        #cv2.imshow('frame1', thresh)
                        previous_thresh_cut = thresh_cut 
                        previous_thresh_original = thresh_original
                        count = count+1
            except:
                pass

            cv2.imshow('frame', output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()

3.0
0.5
30.5
28.5
30.5
16.0
85.5
67.5
47.0
8.5
7.0
3.0
14.5
16.0
105.5
106.5
172.5
261.0
22.0
63.5
16.0
27.0
96.5
105.5
73.5
175.0
144.5
212.5
21.5
24.5
45.0
100.5
50.0
30.5
96.5
95.0
0.0
0.0
62.5
123.0
9.5
0.0
87.5
149.0
100.5
342.5
264.5
594.5
21.5
30.0
8.5
37.0
38.5
9.0
5645.0
242.5
106.5
143.5
20.0
63.5
76.5
167.5
234.5
308.0
130.5
153.0
8.5
20.5
9.0
19.5
300.5
228.0
509.5
537.5
135.5
155.0
172.5
251.0
343.0
500.5
5633.5
122.5
74.5
77.5
43.5
86.5
25.0
66.0
60.0
136.0
92.5
177.0
66.0
100.5
151.0
222.0
3.0
18.5
1.0
7.5
90.0
193.5
184.5
196.0
193.5
210.5
83.5
117.5
22.0
39.5
8.5
25.5
119.5
116.5
375.0
328.5
5733.0
0
5419.5
121.5
1.0
5.0
190.5
143.5
77.0
56.0
111.5
156.5
227.5
229.5
176.5
110.5


In [9]:

# old method 
import numpy as np
import cv2
import pygame

import time


class Playdrum:
    def __init__(self):
        self.area_lim = 61000
        self.area_lim2 = 200
        pygame.init()
        self.start_time = time.time()
        
    def play_sound(self,sound_path):
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        print ('playsound!')
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick(self,image_cut):
        thresh = self.binarize(image_cut,128)
        labeled_image = self.sequential_label(thresh)
        area_lim = 61000
        area_lim2 = 500
        cv2.imshow('frame11', thresh)
        res = self.moment_opencv(labeled_image)
        list_A = []
        for i in res:
            list_A.append(res[i][0])

        if list_A:
            #print ('No stick detected!')
            
            if max(list_A)<self.area_lim:
                #print (max(list_A))
                self.play_sound("drumbeats/drum.wav")

            #print (list_A)
            list_A.pop(list_A.index(max(list_A)))
            if list_A:
                #print ('No stick detected!')
                if max(list_A)>self.area_lim2:
                    #print (max(list_A))
                    self.play_sound("drumbeats/drum_loud.wav")

        return thresh
    def sequential_label(self,binary_in):
        a = binary_in.shape
        a = np.array([int(v) for v in a])
        I = binary_in
        label = 0
        labeled_image = np.zeros((a[0],a[1]))
        record = []
        for i in range(1,a[0]):
            for j in range(1,a[1]):
                if I[i][j]!=0: 
                    if labeled_image[i-1][j-1]!=0:
                        labeled_image[i][j] = labeled_image[i-1][j-1]

                    elif labeled_image[i][j-1]==0 and labeled_image[i-1][j]==0:
                        label = label+1
                        labeled_image[i][j] = label
                    elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]==0:
                        labeled_image[i][j] = labeled_image[i-1][j]

                    elif labeled_image[i-1][j]==0 and labeled_image[i][j-1]!=0:
                        labeled_image[i][j] = labeled_image[i][j-1]    

                    elif labeled_image[i-1][j]!=0 and labeled_image[i][j-1]!=0:

                        if labeled_image[i][j-1]==labeled_image[i-1][j]:
                            labeled_image[i][j] = labeled_image[i][j-1]

                        else: 
                            minValue = min(labeled_image[i][j-1],labeled_image[i-1][j])
                            maxValue = max(labeled_image[i][j-1],labeled_image[i-1][j])
                            labeled_image[i][j] = labeled_image[i][j-1]
                            # record equivalence of labels
                            labeled_image = np.where(labeled_image == minValue, maxValue, labeled_image)

        labelled_img = labeled_image
        # normalized

        max_labelled_img = labelled_img.max()
        labelled_img = (labelled_img*(255/max_labelled_img)).astype('uint8')


        return labelled_img
    def moment_opencv(self,labelled_in):
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res

    
    def run(self):
        cap = cv2.VideoCapture(0)
        count = 0
        scaler = 1.2
        while(True):
            ret, frame = cap.read()
            output = frame
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)

            # ensure at least some circles were found
            if circles is not None:
                # convert the (x, y) coordinates and radius of the circles to integers
                circles = np.round(circles[0, :]).astype("int")

                # loop over the (x, y) coordinates and radius of the circles
                for (x, y, r) in circles:
                    # draw the circle in the output image, then draw a rectangle
                    # corresponding to the center of the circle
                    cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                    cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                    par = np.int64(scaler*r)
                    image_cut = gray[y-par:y+par,x-par:x+par]
                    

                    thresh = self.detect_stick(image_cut)
                    #cv2.imshow('frame1', thresh)

            cv2.imshow('frame', output)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [2]:
#  backup 
import numpy as np
import cv2
import pygame
import time


class Playdrum:
    def __init__(self):
        self.area_lim =1000
        
        self.area_lim2 = 1000
        pygame.init()
        self.start_time = time.time()
        self.first_num = 0
        self.second_num = 0
        self.mask_h = 100
        self.mask_w = 100
        self.take_out = 0
        self.hit_on_count = 0
        self.hit_side_count = 0
        self.p_time = time.time()
        self.n_time = time.time()
        
    def play_sound(self,sound_path):
        drum_beats = pygame.mixer.Sound(sound_path)
        drum_beats.play()
        #print ('playsound!')
        
    def binarize(self,gray_in, threshold=200):
        #ret,thresh1 = cv2.threshold(img,threshold,255,cv.THRESH_BINARY)
        binary_image = np.where(gray_in<threshold,0,255).astype('uint8')
        return binary_image

    def detect_stick_on(self,image_cut):
        cv2.imshow('frame_',image_cut)
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        #print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            print (time_diff)
            print (self.hit_on_count)
            if time_diff> 0.3:
                if self.hit_on_count > 2:
                    self.play_sound("drumbeats/drum.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def detect_stick_side(self,image_cut):
        image_cut = image_cut.astype('uint8')
        cv2.imshow('frame_',image_cut.astype('uint8'))
        image, cnt, hierarchy = cv2.findContours(image_cut.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        area_list = []
        for i in range(len(cnt)):
            area_list.append(cv2.contourArea(cnt[i]))
        max_area = max(area_list)
        #print (max_area)
        if max_area>self.area_lim:
            self.n_time = time.time()
            self.hit_on_count  = self.hit_on_count+1
            time_diff = self.n_time - self.p_time
            print (time_diff)
            print (self.hit_on_count)
            if time_diff> 0.3:
                if self.hit_on_count > 2:
                    self.play_sound("drumbeats/drum.wav")
                    self.hit_on_count = 0
                    self.p_time = time.time()
        return image_cut
    
    def moment_opencv(self,labelled_in):
        res = {}
        for label_idx in np.unique(labelled_in):
            if label_idx == 0:
                continue
            masked_in = (labelled_in==label_idx).astype(labelled_in.dtype)
            moments = cv2.moments(masked_in)
            res[label_idx] = [moments['m00'],moments['m01'],moments['m10'],
                              moments['m02'],moments['m11'],moments['m20'],
                              moments['mu02'],moments['mu11'],moments['mu20']]
        return res
    def createCircularMask(self,h, w, center=None, radius=None):

        if center is None:
            center = [int(w/2), int(h/2)]
        if radius is None:
            radius = min(center[0], center[1], w-center[0], h-center[1])

        Y, X = np.ogrid[:h, :w]
        dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

        mask = dist_from_center <= radius
        return mask
    
    def run(self):
        cap = cv2.VideoCapture(0)
        count = 0
        prievous_thresh_original = 0
        prievous_thresh_cut = 0
        scaler = 1.2
        count = 1
        mask =  self.createCircularMask(self.mask_h, self.mask_w, center=(self.mask_h/2,self.mask_w/2), radius=40)
        
        mask = np.array(abs(1-mask))
        cv2.imshow('frame11',mask.astype('uint8'))
        print (type(mask))
        print (mask.shape)
        print (mask)
        while(True):
            
            ret, frame = cap.read()
            output = frame
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
            
            # ensure at least some circles were found
            if circles is not None:
                # convert the (x, y) coordinates and radius of the circles to integers
                circles = np.round(circles[0, :]).astype("int")

                # loop over the (x, y) coordinates and radius of the circles
                for (x, y, r) in circles:
                    # draw the circle in the output image, then draw a rectangle
                    # corresponding to the center of the circle
                    cv2.circle(output, (x, y), r, (0, 255, 0), 4)
                    cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                    par = np.int64(scaler*r)
                    retval, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
                    thresh_cut = cv2.resize(thresh[y-par:y+par,x-par:x+par],(self.mask_h,self.mask_w))
                    thresh_cut_inv = cv2.bitwise_not(thresh_cut)
                    thresh_original = cv2.resize(thresh[y-r:y+r,x-r:x+r],(self.mask_h,self.mask_w))
                    #cv2.imshow('frame4',thresh_cut)
                    
                    if count !=1:
                        thresh1 = self.detect_stick_side((thresh_cut-previous_thresh_cut)*mask)
                        #thresh1 = self.detect_stick_side(thresh_cut)
                        #thresh2 = self.detect_stick_on(thresh_original-previous_thresh_original)

                    #cv2.imshow('frame3',thresh_original)
                    #cv2.imshow('frame1', thresh)
                    previous_thresh_cut = thresh_cut 
                    previous_thresh_original = thresh_original
                    count = count+1

            cv2.imshow('frame', output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break 
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':
    p = Playdrum()
    p.run()

<class 'numpy.ndarray'>
(100, 100)
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
1.7350425720214844
1
4.009266376495361
2
4.041518688201904
3
1.1669998168945312
1
7.484605312347412
2
12.685030698776245
3
1.0874764919281006
1
1.6626949310302734
2
2.494415760040283
3
0.43213558197021484
1
1.856518030166626
2
1.9202721118927002
3
2.9281983375549316
1
3.7924158573150635
2
4.959145784378052
3
